In [ ]:
\p 8765

In [ ]:
\l /home/iguana/IdeaProjects/qcommon/src/main/q/util/random.q

In [ ]:
rootDir:raze system "pwd"

In [ ]:
dataDir:rootDir,"/data/"

In [254]:
.kin.testTable:.kin.mktrades[`googl`ibm`fb`tmobil`amazn`alibaba;100]

# 14. Introduction to Kdb+

- __[Wikipedia on Kdb+](https://en.wikipedia.org/wiki/Kdb%2B)__

- Kdb+, as well as q, is written in k.
- Kdb+ is the data persistence part of q.
- Kdb+ is a relational database which is:
    - CBDB - column-store (columnar) (algol-like db)
    - TSDB - time series database
    - IMDB - in-memory abilities

- CSDB means faster seeking in data stored on hard disks
- CSDB is slower on retrieving full rows of tables
    - A RSDB retrieves a row in a single disk read (whole row operations are rare)
    - A CSDB retrieves a row in multiple disk operations
- OLTP-focused RDBMSs are more row oriented
- OLAP-focused RDBMSs are a balance of row-oriented and column-oriented
- CSDBs are well suited for OLAP-like workloads (like in data warehouses),
    - where highly complex queries are frequently executed over big data
- CSDBs have been developed as hybrids capable of OLTP and OLAP operations
- RSDBs are better suited for OLTP-like workloads,
    - which are more heavily loaded with interactive transactions.
- Disadvantages of CSDBs to RSDBs:
    - Retrieving entire rows from table
    - Transaction-heavy operations (insert, update, delete)

## 14.1. Tables in memory and serialization

- Persist a table with set and read it into memory with get

### 14.1.1. Tables and key tables

- A table is a trasposed column dictionary, in which address slots are reversed but no data is moved
- The table schema looks like this: ([] s:`symbol$(); v:`int$())
- The type of a table is 98h
- The type of a keyed table is 99h
- The meta commands returns column names, types and attributes

### 14.1.1. Foreign keys and [link colums](https://code.kx.com/v2/wp/the_application_of_foreign_keys_and_linked_columns_in_kdb.pdf)

- Schema for creating a foreign key:

In [ ]:
.kin.table1:([id:til 10] col1:10?`5;col2:3*10?1000)

In [ ]:
.kin.table2:([] t1id:`.kin.table1$5 4 7 2 3;date:.z.d+5?30;percent:90+5?20)

In [ ]:
.kin.table3:([]; t1id:`.kin.table1!(exec id from .kin.table1)?5 4 7 2 3; q:100 101 102 103 109)

In [ ]:
.kin.table1

In [ ]:
.kin.table2

In [ ]:
.kin.table3

- A link column is similar to a foreign key:
    - its entries are indices of rows in a table
    - but you must perform the lookup manually __?!__
- The advantages of link columns:
    - The target can be a table or a keyed table
    - The target can be the table containing the link column
    - Link columns can be splayed or partitioned, whereas foreign keys cannot

In [ ]:
kt:([id:1001 1002 1003] s:`a`b`c; v:100 200 300)
t:([]; id:`kt$1002 1001 1003 1001; q:100 101 102 103)

In [ ]:
q)tk:([] id:1001 1002 100; s:`a`b`c; v:100 200 300)
q)t:([]; id:`tk!(exec id from tk)?1002 1001 1003 1001; q:100 101 102 103)

### 14.1.3. Serializing tables

- __set/get__ to persist and load tables
    - set/get is the the general serialization/deserialization feature of q
    - a single file is created in the OS file system
- \l can also be used instead of get

In [ ]:
(hsym `$dataDir,"table1") set .kin.table1

In [ ]:
.kin.ldtable:get hsym `$dataDir,"table1"

In [ ]:
.kin.ldtable

### 14.1.4. Operating on serialized tables

- To execute an operation on a table you can:
    - either load it into memory:tInMem:get `fileHandle -> select from tInMem
    - or do it directly on the persisted table: select from `fileHandle
- Under the hood, __operations are performed in memory in both cases__!!!
    - Serialized table must fit into memory.

### 14.1.5. The database view

- Zero dimensional persisted form: when 1 table = 1 file
    - The contents of the table is represented by a dot
- Zero dimensional vs higher-dimensional forms: 1 table = multiple files and/or multiple directories.

## Types of data decomposition in q

- Tables can be:
    - __Splayed__: columns of splayed tables are stored in multiple files
    - __Partitioned__: rows of partitiond tables are stored in multiple files
    - __Segmented__: rows of a segmented table are stored in multiple directories (that have the same sturcture as the root directory in a partitioned database)

## 14.2. Spayed tables

- Problems with serialized tables:
    - __Space complexity__: The entire table must fit into memory on each user's machine
    - __Time complexity__ (speed): operations on persisted tables will be slow due to reloading the entire table each time

### 14.2.0. Spaying a table

- Splaying is when the columns of tables is persisted in multiple files in one directory:
    - A splayed tables corresponds to a directory with the name of the table and
    - Each columns corresponds to a file with the name of the column
    - The sym file (.d): the only metadata stored is the list of column names, which is serialized to the hidden sym file
- Splay tables visualization:
    1-dimensional persisted form: a series of dots representing columns
- Splaying resolves the memory/reload issue through mapping the columns to the memory:
    - Columns are loaded into memory on demand then memory is released when no longer needed
- Used case: splay those tables with many columns, since most queries refer to only a handful of columns

### 14.2.1. Creating splayed tables

- To create a splayed table, use
    - a directory as a file handle instead of a file name + set/upsert + tableName or
    - you can splay it manually
- Restrictions on the column types that can be splayed:
    - No general lists: simple or compound lists only
    - Columns of symbol type must be enumerated
    - You cannot splay keyed tables (use link columns instead of keys)
    - Same filename cannot exist as the would-be-splayed directory name
- To read a splayed table, use
    - get + the original directory handle
- To read only a column, use:
    - get + column file name as a file handle
- To list the files in the directory:
    - \ls + the full path __without__ the backtick and the backslash at the end

In [ ]:
.kin.dirHandle:hsym `$dataDir,"testTrades/"

In [ ]:
.kin.dirFileName:hsym `$dataDir,"testTrades" / this won't work with \ls

In [ ]:
.kin.dirHandle set .kin.spTrades

In [ ]:
.kin.dirFileName set .kin.mktrades[`goog`ibm`fb`tmobil`amazn`alibaba;100]

In [ ]:
key `.kin

In [ ]:
.kin.mktrades

In [ ]:
.kin.tl:get .kin.dirHandle

In [ ]:
\ls -a /home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/testTrades

### 14.2.2. Spayed tables with symbol columns

- For all splayed and partitioned tables involved in spalying, all symbol columns must be enumerated over the list of symbols that is serialized into the root directory.
- For enumerating, use the projected version of built-in .Q.en function:

In [ ]:
`:/db/t/ set .Q.en[`:/db;] ([] s1:`a`b`c; v:10 20 30; s2:`x`y`z)

### 14.2.3. Splayed tables with nested columns

- The only type of nested colulmns that canbe splayed is made up of compound lists:
    - compound list is a list of simple lists of uniform type
    - compund lists are indicated by upper case letters in the type column of meta results

In [ ]:
/ function to examine if a column can be splayed
canBeSplayed:{where {(ts~1#0h)|1<count ts:distinct `type each`x} each flip x}

### 14.2.4. Basic operations on splayed tables

- Map a splayed table to memory by
    - either loading it on startap: $$q path/to/splayedTableDir
    - or \l /path/to/splayedTableDir
        - only this format works:
            - no string,
            - no file handle,
            - no trailing fwSlash
            - no tilde instead of /home/userName
- In these cases, tables are only mapped to memory, they are not loaded until an expression is evaluated on them
- \a list tables in current context
- extract column data via symbol indexting (dot notation does not work): tableName ``colName
- Operations that work on splayed tables:
    - select and exec (exec does not work on partitiond tables)

In [251]:
.kin.dirHandle set .kin.mktrades[`googl`ibm`fb`tmobil`amazn`alibaba;100]

`:/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/testTrades/


In [253]:
testTrades `time

04:12:55.527 06:03:01.029 01:11:42.896 09:21:44.893 17:12:24.021 21:11:04.805..


### 14.2.5. Operations on a splayed directory

- select
- exec
- upsert
- xasc
- ``attr#
- updates applied to a mapped table are only applied in the memory not on the disk -> updates are not persistant
    - and there are no built-in operations to update data in a persisted table, only workarounds

### 14.2.6. Appending to a splayed table

- User upsert to append records to a splayed table:
    - directoryHandle + upsert + table2Name (that contains the rows to be appended)

### 14.2.7. Manual operations on a splayed directory

- This is only file system operations -> inconsistency may occur:
    - Do this only when no queries are executed


In [ ]:
`:/db/t/ set ([] ti:09:30:00 09:31:00; p:101.5 33.5)
`:/db/t/p set .[get `:/db/t/p; where 09:31:00=get `:/db/t/ti; :;42.0] / replace item at depth

- Adding a new column to a splayed table:

In [ ]:
`:/db/t/s set (count get `:/db/t/ti)#`
`:/db/t/.d set get[`:/db/t/.d] union `s

- To delete a column, remove the column file and revise the .d file:

In [ ]:
system "rm /db/t/s"
`:/db/t/.d set get[`:/db/t/.d] except `s

- Sorting a splayed table. E.g.:
    - create sort index (iasc, idesc)
    - re-index all column files

In [ ]:
cs:system "ls /db/t"
I:idesc `:/db/t/ti
{pth set get[pth:hsym `$"/db/t/",x] I} each cs

### 14.2.8. Working with sym files

- What can be done by manupulating the sym files?
    1. Moving a table from one database to another
    2. Change column type from symbol to string
    3. Consolidating enumeration domains
- The sym file can easily be corrupted: always backup database before modifying the sym file

1. Moving tables between databases:
- un-enumerate table in source database
- re-enumerate table in target database

### 14.2.9. Splayed tables with link columns

### 14.2.10. Query execution on splayed tables

- When the same column is used in separate queries, it is loaded into memory only once and cached in memory until it is garbage collected.

## 14.3. Partitioned tables

- Table partitioning is the 2nd type of table decomposing in q. It is useful when the database is so large even one column cannot fit into the memory.
- E.g.: slice columns into daily partitions
- __ALL PARTITIOND TABLES ARE SPLAYED__, but not all splayed table are partitioned

### 14.3.1. Partitions

- Partitioning a splayed table is further decomposing it by grouping records having common values along a column of a special type.
- Along columns of only this special type can partitioning be made.
- The special type is any type that has an integer under the covers:
    - boolean, 0b-1b, 1, ``boolean
    - short
    - int
    - long
    - char
    - timestamp
    - date
    - timespan

- A partitioned table is 2-dimensional persisted form, it is cut horizontally and vertically:
- Partitions are stored in separate directories in the root directory
- and each partition-directory contains a subdirectory with the name of the table, which contains the splayed files:
/root
    /partitionvalue1
        /tablename
            .d
            column1name
            column2name
            …
    /partitionvalue2
        /tablename
            .d
            column1name
            column2name
            …
        …

- Kdb creates a virtual column for the partition variable. Its name cannot be controlled

### 14.3.2. Partition domain

- Kdb can have only a single partition domain
- If you need different granularity of a table, you need to create two separate databases
- How to partition the table?
    - Choose partition granularity based on what kind of units are most frequently queried

### 14.3.3. Creating partitioned tables

### 14.3.4. Working with partitioned tables

- You cannot do these operations on a partitioned table:
    - retreive an entire row or a column (for obvious reasons)

- Exec does not work on partitioned tables, but there is a workaround:
    - exec … from select … from … 

- Always place the partition constraint first in a where clause with multiple conditions

### 14.3.5. The virtual column i in partitioned tables

- virtual column i refers to the relative row number within a partition

### 14.3.6. Qurey execution on partitioned tables

- Columns are loaded into memory as follows:
    - kdb analyses the where phrase to determine which partitions are targeted by the query
    - processes the remaining where phrases the subdomains that must be loaded
    - processes the query separately against the requisite partitions to obtain partial results
    - combines the partial results to obtain the final result
- You can speed up the query execution by starting q with slaves. In this case the query will be executed concurrently, one partition per slave.

### 14.3.7. Map-reduce

- MISSING

### 14.3.8. Multiple partitioned tables

- MISSING

### 14.3.9. Examples of other partition domain types

- MISSING

### 14.3.10. Partitioned tables with links

- MISSING

## 14.4. Segmented tables

- Performance gain through partitioning can only be achieved when the queries are not I/O-bound.
- To speed up I/O-bound queries, the solution requires multiple I/O channels, so data retreival and processing can occur in parallel.
- The third form of data decomposition, segmentation solves exactly this problem

- Table segmentation is an additional level on top of partitioning
- Segmentation spreads a partitiond table's records across multiple directories that have the same sturcture as the root directory in a partitioned database
- Each pseudo-root is called a segment, which are directories that contain a collection of partition directories.
- The segment directories are on independent I/O channels so that data retreival can occur in parallel.
- Any criteria can be used to decompose partitions into segments as long as results are conforming record subsets that are disjoint and complete (they reconstitute the original table with no omissions or duplication).
- Segmentation can happen
    - along rows,
    - partitions or
    - a combination of them
    - but cannot occur only along cloumns

- A segmented table has a 3-dimensional persisted form:
    - cut vertically by splaying
    - cut horizontally by partitioning
    - cut across physical locations

- The segment directories must not reside under the root
- Only the sym file and the par.txt can reside in the root dirextory:
    - the par.txt contains the paths of the physical locations of the segments

/db
    [sym]
    par.txt
=============== <- channel 1
/segment1
    /partition*
        /table*
        /table*
        …
    /partition*
        /table*
        /table*
        …
=============== <- channel 2
/segment2
    /partition*
        /table*
        /table*
        …
    /partition*
        /table*
        /table*
        …
=============== …

- Example for segmantation: segment the table by bucketing trades into laternating days of the week:

/1                  <- drive 1
    /2015.01.01
        /t          <- day’s trades
    /2015.01.03
        /t          <- day’s trades
    …
=============
/2                  <- drive 2
    /2015.01.02
        /t          <- day’s trades
    /2015.01.04
        /t          <- day’s trades
    …

- E.g.2: segment by alphabet: symbols from a-m and n-z

### 14.4.2. Segmentation vs. partitions

[SEE COMPARISON MATRIX HERE](https://code.kx.com/q4m3/14_Introduction_to_Kdb+/#1442-segmentation-vs-partitions)

### 14.4.3. Creating segmented tables

- MISSING: see examples here

### 14.4.4. Multiple segmented tables

- MISSING: The segmentation of several tables that share a partition

### 14.4.5. Query execution against segmented tables

- Design principles for segmentations:
    1. Maximize the number of independent I/O channels to retrieve data in parallel: n (several hdds or sdds)
    2. Maximize server memory to allocate each slave thread as much memory as it needs
    3. Create n segments to spread data retrieval across the n I/O channels
    4. Open at least n slave threads.

- In such an environment, kdb docomposes a query into two steps of mapping and reducing:
    - Map: a revised form of the original query that executes on each segment
    - Reduce: aggregate the segment results
- This results in preliminary calculations s close to the data as possible while performing the aggregation centrally at hte last step.

- MISSING

### 14.4.6. Balancing slaves and cores

- MISSING

### 14.4.7. Sample performance data

- MISSING

## 14.5. Utilities for splaying and partitioning

## 14.6. Kdb database

## 14.7. Putting it all together

## 14.8. Envitonment variables (QHOME, QLIC, QINIT)

- Three environment variables QHOME, QLIC and QINIT are used by kdb+ at startup.
- __QHOME__ specifies the directory where kdb+ expects to find the bootstrap file q.k. By default, it also looks there for the license file k4.lic. If QHOME is not defined, kdb+ falls back to $HOME/q for Unix-based systems and c:\q for Windows.
- __QLIC__ overrides the default location for the license file. If QLIC is not defined, kdb+ falls back to QHOME (or its fallback).
- __QINIT__ specifies the name of the file that is executed immediately after the load of q.k. If QINIT is not defined, kdb+ attempts to load the file __q.q__ from QHOME. If QHOME is not defined or q.q is not found, no error is reported.